In [1]:
import numpy as np
import os
import scipy.stats

In [2]:
target_path = 'test_mc_bugfree'

os.chdir(target_path)

prots = []
algs = []
data = {}

for f in os.listdir():
    if os.path.isdir(f) and len(f) == 4:
        print('Found %s' % f)
        prots.append(f)
        os.chdir(f)
        data[f] = {}
        for ff in os.listdir():
            if os.path.isfile(ff) and 'raw' in ff and '10' not in ff:
            #if os.path.isfile(ff) and 'raw' in ff:
                base_name = ff[5:-8]
                if base_name not in algs:
                    algs.append(base_name)
                data[f][base_name] = {'rmsd': [], 'score': []}
                with open(ff) as datafile:
                    for line in datafile.readlines():
                        tokens = line.strip().split(' ')
                        score = float(tokens[1])
                        rmsd = float(tokens[5])

                        data[f][base_name]['rmsd'].append(rmsd)
                        data[f][base_name]['score'].append(score)
        os.chdir('..')

os.chdir('..')

os.chdir('/home/h3nnn4n/progs/de_supimpa/src/de')

algs = sorted(algs, key=len)

Found 1ail
Found 1zdd
Found 1enh
Found 1crn


In [3]:
ignore_prots = True

mode = 'score'
#mode = 'rmsd'

for prot in prots:
    comp = []
    for alg in algs:
        comp.append(data[prot][alg][mode])

In [4]:
print(scipy.stats.kruskal(*comp))
print(scipy.stats.f_oneway(*comp))

KruskalResult(statistic=63.754496314496237, pvalue=7.0107133110784886e-10)
F_onewayResult(statistic=16.828752888714828, pvalue=2.2317543495994304e-17)


In [5]:
alpha = 0.05

results = {}

print('---------------------- EQUIVALENT TO ----------------------')

for k, i in enumerate(comp):
    print('%25s' % algs[k], end=': ')
    for l, j in enumerate(comp):
        if k == l:
            continue
        aa = len(i)
        if len(i) != len(j):
            aa = min(len(i), len(j))
        _, p = scipy.stats.wilcoxon(i[:aa], j[:aa]) 
        if p >= alpha:
            print('%s' % algs[l], end=' ')    
    print()

print('\n---------------------- BETTER THAN ----------------------')

for k, i in enumerate(comp):
    results[algs[k]] = 0
    print('%25s' % algs[k], end=': ')
    for l, j in enumerate(comp):
        if k == l:
            continue
        aa = len(i)
        if len(i) != len(j):
            aa = min(len(i), len(j))
        _, p = scipy.stats.wilcoxon(i[:aa], j[:aa]) 
        # _, p = scipy.stats.ranksums(i, j) 
        if p < alpha:
            if np.mean(i) < np.mean(j):
                results[algs[k]] += 1
                print('%s' % algs[l], end=' ')               
    print()
    
print('\n---------------------- N best ----------------------')

for k, i in enumerate(comp):
    print('%25s: %2d'% (algs[k], results[algs[k]]))
    
print('\n---------------------- CONFIDENCE INTERVAL ----------------------')
for k, a in enumerate(comp):
    ic = scipy.stats.t.interval(0.9, len(a) - 1, loc=np.mean(a), scale=scipy.stats.sem(a))
    print('%25s: %8.4f %8.4f' % (algs[k], ic[0], ic[1]))

---------------------- EQUIVALENT TO ----------------------
                  sade_ss: sade_de sade_de_ss 
                  sade_de: sade_ss 
                sade_frag: sade_de_frag_ss sade_best1exp_frag sade_currToBest_frag 
               sade_de_ss: sade_ss 
             sade_frag_ss: sade_de_frag sade_de_frag_ss sade_best1exp_frag sade_currToBest_frag sade_best1exp_frag_ss sade_currToBest_frag_ss 
             sade_de_frag: sade_frag_ss sade_de_frag_ss sade_best1exp_frag sade_best1exp_frag_ss sade_currToBest_frag_ss 
          sade_de_frag_ss: sade_frag sade_frag_ss sade_de_frag sade_best1exp_frag sade_currToBest_frag sade_best1exp_frag_ss sade_currToBest_frag_ss 
       sade_best1exp_frag: sade_frag sade_frag_ss sade_de_frag sade_de_frag_ss sade_currToBest_frag sade_best1exp_frag_ss sade_currToBest_frag_ss 
     sade_currToBest_frag: sade_frag sade_frag_ss sade_de_frag_ss sade_best1exp_frag sade_best1exp_frag_ss sade_currToBest_frag_ss 
    sade_best1exp_frag_ss: sade_frag_ss sad

In [ ]:
11 x 4 x 10
1231231233 98098098098 7987987987 75688765786 